In [2]:
import pandas as pd
from sklearn.mixture import GaussianMixture # for GMM clustering
from sklearn import metrics # for calculating Silhouette score

import matplotlib.pyplot as plt # for data visualization
import plotly.express as px  # for data visualization
import plotly.graph_objects as go # for data visualization
from sklearn.cluster import KMeans
import pandas as pd
import plotly
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import timeit
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
import warnings
warnings.filterwarnings(action='once')

Helper fucntions 

In [19]:
def modelfit(estimator,data_train,data_test,predictors,target):
    start = timeit.default_timer()
    #fitting model
    estimator.fit(data_train[predictors],data_train.loc[:,target])
    #train data prediction
    start1 = timeit.default_timer()
    train_pred=estimator.predict(data_train[predictors])
    stop1 = timeit.default_timer()
    #cross_validation score
    cv_score=cross_val_score(estimator,data_train[predictors],data_train.loc[:,target],cv=5,scoring='precision')

    #Print model report:
    print ("\nModel Report")
    print ("Accuracy on Train Data: %.4g" % metrics.accuracy_score(data_train.loc[:,target].values, train_pred))
    print ("Precision on Train Data: %.4g" % metrics.precision_score(data_train.loc[:,target].values, train_pred))
    print ("Recall on Train Data: %.4g" % metrics.recall_score(data_train.loc[:,target].values, train_pred))
    print ("F1 score on Train Data: %.4g" % metrics.f1_score(data_train.loc[:,target].values, train_pred))
    
    print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    start2 = timeit.default_timer()
    test_pred=estimator.predict(data_test[predictors])
    stop2 = timeit.default_timer()
    print ("Accuracy on Test Data: %.4g" % metrics.accuracy_score(data_test.loc[:,target].values, test_pred))  
    print ("Precision on Test Data: %.4g" % metrics.precision_score(data_test.loc[:,target].values, test_pred)) 
    print ("Recall on Test Data: %.4g" % metrics.recall_score(data_test.loc[:,target].values, test_pred)) 
    print ("F1 score on Test Data: %.4g" % metrics.f1_score(data_test.loc[:,target].values, test_pred)) 
    stop = timeit.default_timer()
    print('Total Time: ', stop - start)
    print('training time: ', stop1 - start1)
    print('testing time: ', stop2 - start2)
    return test_pred

# Credit card data problem 

## Clustering 

In [30]:
df = pd.read_csv('../Data/CC_df_sampled.csv')
X_train, X_final_test = train_test_split(df,test_size=0.2, random_state= 2)
X_final_test.to_csv('../Data/testing_dataset.csv', index = False)
## within the training dataset, split the data into training dataset and testing/validation dataset
X_train, X_validation = train_test_split(X_train,test_size=0.2, random_state= 2)

In [31]:
X_all = df.drop(['approved', 'user_id'], axis = 1)
y_all = df['approved']
X = X_train.drop(['approved', 'user_id'], axis = 1)
y = X_train['approved']
X_val = X_validation.drop(['approved', 'user_id'], axis = 1)
y_val = X_validation['approved']
X_final = X_final_test.drop(['approved', 'user_id'], axis = 1)
y_final = X_final_test['approved']

### GMM 

Based on prior research, GMM will have 2 clusters

### K means

Based on prior research, K means will have 7 clusters

## Clustering Results Evaluation 

### GMM

all of the predictors from the original training dataset have been converted to cluster probabilities 

In [33]:
GMM_model = GaussianMixture(n_components=2, # this is the number of clusters
                         covariance_type='full', # {‘full’, ‘tied’, ‘diag’, ‘spherical’}, default=’full’
                         max_iter=100, # the number of EM iterations to perform. default=100
                         n_init=20, # the number of initializations to perform. default = 1
                         init_params='kmeans', # the method used to initialize the weights, the means and the precisions. {'random' or default='k-means'}
                         verbose=0, # default 0, {0,1,2}
                         random_state=1 # for reproducibility
                        ).fit(X)
#y_pred = GMM_model.fit_predict(X)
X_prob = GMM_model.predict_proba(X)

X_prob = pd.DataFrame(X_prob, columns = ['cluster1', 'cluster2'])
X_prob['target'] = list(y)

In [26]:
X.shape

(6400, 68)

X_validation & X_final should also be transformed into clustering probs as predictors

In [34]:
X_val_prob = GMM_model.predict_proba(X_val)

X_val_prob = pd.DataFrame(X_val_prob, columns = ['cluster1', 'cluster2'])

X_val_prob['target'] = list(y_val)

X_final_prob = GMM_model.predict_proba(X_final)

X_final_prob = pd.DataFrame(X_final_prob, columns = ['cluster1', 'cluster2'])

X_final_prob['target'] = list(y_final)

In [38]:
X_val_prob.head()

,cluster1,cluster2,target
0,1.000000e+00,0.0,0
1,2.306663e-28,1.0,0
2,6.929901e-34,1.0,1
3,1.000000e+00,0.0,0
4,3.330747e-33,1.0,0


In [35]:
predictors = ['cluster1', 'cluster2']
target = 'target'

In [44]:
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('always')
mlp = MLPClassifier(random_state=1)
y_pred=modelfit(mlp, X_prob, X_val_prob,predictors,target)

/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classi


Model Report
Accuracy on Train Data: 0.755
Precision on Train Data: 0
Recall on Train Data: 0
F1 score on Train Data: 0
CV Score : Mean - 0 | Std - 0 | Min - 0 | Max - 0
Accuracy on Test Data: 0.7481
Precision on Test Data: 0
Recall on Test Data: 0
F1 score on Test Data: 0
Total Time:  1.9832877489998282
training time:  0.005590953000137233
testing time:  0.002480572999957076


/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
X_val_prob['y_pred'] = y_pred

In [41]:
X_val_prob.describe()

,cluster1,cluster2,target,y_pred
count,1.600000e+03,1600.000000,1600.000000,1600.0
mean,1.606250e-01,0.839375,0.251875,0.0
std,3.672995e-01,0.367300,0.434226,0.0
min,0.000000e+00,0.000000,0.000000,0.0
25%,8.756579e-46,1.000000,0.000000,0.0
50%,4.707600e-32,1.000000,0.000000,0.0
75%,6.440923e-28,1.000000,1.000000,0.0
max,1.000000e+00,1.000000,1.000000,0.0


In [54]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'activation': ['tanh', 'relu'],
              'alpha': [0.0001, 0.1],
              'hidden_layer_sizes': [(100,),(500,)],
               'learning_rate_init': [0.001, 0.01]}
grid_lr = GridSearchCV(estimator =mlp, param_grid= param_grid, scoring='precision', 
                        refit=True, return_train_score=True) 
grid_lr.fit(X_prob[['cluster1', 'cluster2']], X_prob['target'])

GridSearchCV(estimator=MLPClassifier(random_state=1),
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.0001, 0.1],
                         'hidden_layer_sizes': [(100,), (500,)],
                         'learning_rate_init': [0.001, 0.01]},
             return_train_score=True, scoring='recall')

In [56]:
mlp_tuned = grid_lr.best_estimator_
print(mlp_tuned.get_params())

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [57]:
y_pred=modelfit(mlp_tuned, X_prob, X_val_prob, predictors,target)

/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classi


Model Report
Accuracy on Train Data: 0.755
Precision on Train Data: 0
Recall on Train Data: 0
F1 score on Train Data: 0
CV Score : Mean - 0 | Std - 0 | Min - 0 | Max - 0
Accuracy on Test Data: 0.7481
Precision on Test Data: 0
Recall on Test Data: 0
F1 score on Test Data: 0
Total Time:  2.505461706999995
training time:  0.00974361000044155
testing time:  0.0035320489996593096


/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### K Means

In [69]:
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
def transform_data_w_K_means(X,y):
    y_pred = kmeans.predict(X)
    X_cluster = kmeans.fit_predict(X)
    ##distance from centroid
    X_dist = kmeans.transform(X)
    X_kmeans = pd.DataFrame()
    X_kmeans['cluster'] = list(X_cluster)
    X_kmeans = pd.get_dummies(X_kmeans, columns = ['cluster'])
    X_kmeans['distance'] = list(X_dist)
    X_kmeans['distance'] = X_kmeans['distance'].apply(lambda x: x[0])
    X_kmeans['target'] = list(y)
    return X_kmeans

In [70]:
X_kmeans = transform_data_w_K_means(X, y)
X_val_kmeans = transform_data_w_K_means(X_val, y_val)
X_final_kmeans = transform_data_w_K_means(X_final, y_final)


In [84]:
X_final_kmeans.head()

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,distance,target
0,1,0,0,0,0,0,0,3.281337,0
1,1,0,0,0,0,0,0,3.254721,0
2,1,0,0,0,0,0,0,8.303704,0
3,1,0,0,0,0,0,0,4.734045,1
4,0,0,1,0,0,0,0,5453.025820,0


In [87]:
predictors = ['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'distance']
target = 'target'

In [88]:
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('always')
mlp = MLPClassifier(random_state=1)
y_pred=modelfit(mlp, X_kmeans, X_val_kmeans, predictors,target)


Model Report
Accuracy on Train Data: 0.767
Precision on Train Data: 0.7692
Recall on Train Data: 0.07015
F1 score on Train Data: 0.1286
CV Score : Mean - 0.7803 | Std - 0.1376 | Min - 0.6111 | Max - 1
Accuracy on Test Data: 0.2325
Precision on Test Data: 0.2051
Recall on Test Data: 0.7122
F1 score on Test Data: 0.3185
Total Time:  2.9856333219941007
training time:  0.0050738539939629845
testing time:  0.0024131189929903485


In [80]:
from sklearn.model_selection import GridSearchCV
param_grid = { 'activation': ['tanh', 'relu'],
              'alpha': [0.0001, 0.1],
              'hidden_layer_sizes': [(100,),(500,)],
               'learning_rate_init': [0.001, 0.01]}
grid_lr = GridSearchCV(estimator =mlp, param_grid= param_grid, scoring='precision', 
                        refit=True, return_train_score=True) 
grid_lr.fit(X_kmeans[predictors], X_kmeans['target'])

/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ej7793/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classi

GridSearchCV(estimator=MLPClassifier(random_state=1),
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.0001, 0.1],
                         'hidden_layer_sizes': [(100,), (500,)],
                         'learning_rate_init': [0.001, 0.01]},
             return_train_score=True, scoring='precision')

In [82]:
mlp_tuned = grid_lr.best_estimator_
print(mlp_tuned.get_params())

{'activation': 'tanh', 'alpha': 0.1, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_fun': 15000, 'max_iter': 200, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [83]:
y_pred=modelfit(mlp_tuned, X_kmeans, X_final_kmeans, predictors,target)


Model Report
Accuracy on Train Data: 0.7625
Precision on Train Data: 0.8158
Recall on Train Data: 0.03954
F1 score on Train Data: 0.07543
CV Score : Mean - 0.8675 | Std - 0.1193 | Min - 0.6667 | Max - 1
Accuracy on Test Data: 0.23
Precision on Test Data: 0.1988
Recall on Test Data: 0.6786
F1 score on Test Data: 0.3076
Total Time:  2.6800296310002523
training time:  0.006921193999914976
testing time:  0.00445235099959973
